In [2]:
import dask.dataframe as dd

import boto3
from os import path, makedirs, remove
from datetime import datetime
import logging
from logging import Logger
import pandas as pd

In [23]:
# S3 bucket and paths
BUCKET_NAME = "oedi-data-lake"

PARENT_PREFIX = "pvdaq/parquet/"
SITE_PREFIX = PARENT_PREFIX + "site/"
MOUNT_PREFIX =  PARENT_PREFIX + "mount/"

METRICS_PREFIX = PARENT_PREFIX + "metrics/metrics__system_{ss_id}"
PV_PREFIX = PARENT_PREFIX + "pvdata/system_id={ss_id}/year={year}/month={month}/day={day}/"

In [2]:
class PVTransform:
    def __init__(self, staging_area: str):
        self.staging_area = staging_area


    def join_metadata(self):
        metadata_cols = ["system_id", "latitude", "longitude", "elevation", "av_pressure", "av_temp", "climate_type"]
        metadata = pd.read_parquet(path.join(self.staging_area, "site.parquet"), columns=metadata_cols)

        mount_cols = ["system_id", "azimuth", "tilt"]
        mount_data = pd.read_parquet(path.join(self.staging_area, "mount.parquet"), columns=mount_cols)

        merged = pd.merge(metadata, mount_data, "inner", "system_id").rename({"system_id":"ss_id", "azimuth":"mount_azimuth", "tilt":"mount_tilt"}, axis=1)
        merged.to_parquet(path.join(self.staging_area, "metadata.parquet"))
        
        remove(path.join(self.staging_area, "site.parquet"))
        remove(path.join(self.staging_area, "mount.parquet"))
    
    

In [69]:
transformer = PVTransform(staging_area="./data/")

In [72]:
transformer.join_metadata()

In [3]:
pv_cols = ["measured_on", "metric_id", "value"]
pv_data = dd.read_parquet(path='data/system_1349/pv_data/*.parquet', columns=pv_cols)

In [6]:
pv_data["ss_id"] = 1349